![PPGI_UFRJ](https://github.com/zavaleta/Fundamentos_DS/blob/main/imagens/ppgi-ufrj.png?raw=1)
# **Fundamentos de Ciência de Dados**

> Trabalho Final - 2º Período de 2022

### **Disponibilizando Dados sobre Resultados Financeiros de Cias Abertas Enriquecidos com Proveniência para a [OBInvest](https://obinvest.org/)**

---

**Alunos:** Gilberto Gil | Saulo Andrade Almeida | Valquire Jesus



## **FAIRificação**

In [174]:
#checking version machine architecture, OS, python and all libs used in this notebook
import platform as platform
import numpy as np
import pandas as pd
import os
import conda
import pydot
import prov

def checkingEnvironmentVersions(details=False):

    #definnig version of python and all libs used
    HOST_MACHINE_ARCHTECTURE_EXPECTED = 'x86_64'
    HOST_MACHINE_OS_EXPECTED = 'Linux'
    CONDA_VERSION_EXPECTED = '4.14.0'
    CONDA_DEFAULT_ENV_EXPECTED = 'drecvmenv'
    PYTHON_VERSION_EXPECTED = '3.9.12'
    NUMPY_LIB_VERSION_EXPECTED = '1.21.5'
    PANDAS_LIB_VERSION_EXPECTED = '1.4.2'
    PYDOT_LIB_VERSION_EXPECTED = '1.4.2'
    PROV_LIB_VERSION_EXPECTED = '2.0.0'
    

    if details: 
        print('Host Machine Architecture:', platform.machine())
        print('Host Machine OS:', platform.system())
        print('Conda Version:', conda.__version__)
        print('Conda default env:', os.environ['CONDA_DEFAULT_ENV'])
        print('Python Version:', platform.python_version())
        print('NumPy Lib Version:', np.__version__)
        print('Pandas Lib Version:', pd.__version__)
        print('PyDot Lib Version:', pydot.__version__)
        print('Prov Lib Version:', prov.__version__)
        
    #checking versions
    try:
        #checking Machine Architecute expected
        assert platform.machine() == HOST_MACHINE_ARCHTECTURE_EXPECTED

        #checking OS expected
        assert platform.system() == HOST_MACHINE_OS_EXPECTED
        
        #checking conda version
        assert conda.__version__ == CONDA_VERSION_EXPECTED
        
        #checking conda default environment
        assert os.environ['CONDA_DEFAULT_ENV'] == CONDA_DEFAULT_ENV_EXPECTED    

        #checking python version
        assert platform.python_version() == PYTHON_VERSION_EXPECTED

        #checking numpy lib version
        assert np.__version__ == NUMPY_LIB_VERSION_EXPECTED  

        #checking Pandas lib version
        assert pd.__version__ == PANDAS_LIB_VERSION_EXPECTED
        
        #checking pydot version
        assert pydot.__version__ == PYDOT_LIB_VERSION_EXPECTED
        
        #checking prov version
        assert prov.__version__ == PROV_LIB_VERSION_EXPECTED
    except:
        #if any assert fail, or something else get wrong during verification
        if details: print('Something is wrong!')
        return False
    else:
        #if pass all asserts
        if details: print('All versions are correct!')
        return True

## **Pré-processamento de Dados**

In [175]:
#Utilizado ambiente python 3.9
import pandas as pd
import numpy as np

def carrega_csv(path, nomes_colunas_data=[]):
    return pd.read_csv(path, sep=';', header=0, encoding="ISO-8859-1", parse_dates=nomes_colunas_data)

def carrega_csvs_dre(template_path_arquivo, anos):
    tempDF = pd.DataFrame() 
    
    for ano in anos:
        df_ano_corrente = carrega_csv(template_path_arquivo.format(ano), 
                                      ['DT_REFER', 'DT_INI_EXERC', 'DT_FIM_EXERC'])
        
        tempDF = pd.concat([tempDF, df_ano_corrente] , ignore_index=True)
        
    return tempDF
    

def carregar_dados_empresas():
    return carrega_csv("data/cad-emp/cad_cia_aberta.csv")

def lista_todos_anos():
    #return [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
    return [2020, 2021]

def carrega_todos_anos_dre_itr():
    return carrega_csvs_dre('data/dre-itr/itr_cia_aberta_DRE_con_{0}.csv', lista_todos_anos())

def carrega_todos_anos_dre_dfp():
    return carrega_csvs_dre('data/dre-dfp/dfp_cia_aberta_DRE_con_{0}.csv', lista_todos_anos())

def carregar_datasets():
    companyInfoDF = carregar_dados_empresas()
    dreItrDF = carrega_todos_anos_dre_itr()
    dreDfpDF = carrega_todos_anos_dre_dfp()
    return companyInfoDF, dreItrDF, dreDfpDF

def trata_dados_companhias(dataframe_cia):
    # TRATAMENTO DO DF CAD_CIA
    # filtrar as empresas com cadastro ativo - df cad_cia
    dataframe_cia = dataframe_cia[dataframe_cia['SIT'] == 'ATIVO']
    # deletar colunas desnecessárias
    dataframe_cia = dataframe_cia.drop(['DENOM_SOCIAL', 'DENOM_COMERC', 'DT_REG', 'DT_CONST', 'DT_CANCEL', 'MOTIVO_CANCEL', 'SIT', 
                   'DT_INI_SIT', 'TP_MERC', 'CATEG_REG', 'DT_INI_CATEG', 'SIT_EMISSOR', 'DT_INI_SIT_EMISSOR', 
                   'CONTROLE_ACIONARIO', 'TP_ENDER', 'LOGRADOURO', 'COMPL', 'BAIRRO', 'MUN', 'UF', 'PAIS', 
                   'CEP', 'DDD_TEL', 'TEL', 'DDD_FAX', 'FAX', 'EMAIL', 'TP_RESP', 'RESP', 'DT_INI_RESP', 
                   'LOGRADOURO_RESP', 'COMPL_RESP', 'BAIRRO_RESP', 'MUN_RESP', 'UF_RESP', 'PAIS_RESP', 
                   'CEP_RESP', 'DDD_TEL_RESP', 'TEL_RESP', 'DDD_FAX_RESP', 'FAX_RESP', 'EMAIL_RESP', 
                   'CNPJ_AUDITOR', 'AUDITOR'], axis=1)
    
    # deletar os registros duplicados, que possuem mesmos cnpj e códigos cvm
    dataframe_cia = dataframe_cia.drop_duplicates(subset = ['CNPJ_CIA', 'CD_CVM'], keep = 'last')
    
    #TODO:Gerar proveniencia do tratamento de companhias
    return dataframe_cia

def adiciona_ano(dataframe_dre):
    ano = []
    
    #adicionando as informacoes da coluna de Ano
    for i in dataframe_dre.itertuples():
        ano.append(i.DT_REFER.year)
    
    dataframe_dre['ANO'] = ano
    
    return dataframe_dre

def adiciona_trimestre(dataframe_dre):
    trimestre =[]
    
    #adicionando as informacoes da coluna de trimestre
    for i in dataframe_dre.itertuples():
      if i.DT_REFER.month == 3:
        trimestre.append(1)
      elif i.DT_REFER.month == 6:
        trimestre.append(2)
      elif i.DT_REFER.month == 9:
        trimestre.append(3)
      else:
        trimestre.append(4)
    
    dataframe_dre['TRIMESTRE'] = trimestre
    
    return dataframe_dre

def merge_dre_setor(dataframe_dre, dataframe_cia):
    # filtrar os informes da empresas ativas
    dataframe_dre = pd.merge(dataframe_dre, dataframe_cia, how = 'inner', on = ['CNPJ_CIA', 'CD_CVM']).reset_index(drop=True)
    return dataframe_dre.drop(['VERSAO'], axis=1)  # eliminar coluna desnecessária
        
def filtrarUltimoExerc(dre):
    # Descartando resultados do ano anterior e mantendo apenas o último valor 
    return dre[dre['ORDEM_EXERC'] == 'ÚLTIMO']
    
def obter_trimestres_123(dataframe_itr):
    # FATIAMENTO DO DF ITR
    dataframe_itr = filtrarUltimoExerc(dataframe_itr)
    # OBTENÇÃO DO DF TRIMESTRES 1, 2 E 3
    # criar o df com informações dos trimestres 1 (1 a 3 mês), 2 (4 a 6 mês) e 3 (6 a 9 mês) - df trim123
    return dataframe_itr.loc[lambda dataframe_itr: ((dataframe_itr.DT_REFER.dt.month == 3) | 
                            ((dataframe_itr.DT_REFER.dt.month == 6) & (dataframe_itr.DT_INI_EXERC.dt.month > 3)) | 
                            ((dataframe_itr.DT_REFER.dt.month == 9) & (dataframe_itr.DT_INI_EXERC.dt.month > 6))) &
                             (dataframe_itr.DT_REFER.dt.year == dataframe_itr.DT_INI_EXERC.dt.year)]

def obter_acumulado_trim3(dataframe_itr):
    # criar o df com informações acumuladas até o trimestre 3, 01/07/ANO a 30/09/ANO - df_acm3
    return dataframe_itr.loc[lambda dataframe_itr: ((dataframe_itr.DT_REFER.dt.month == 9) & 
                                                   (dataframe_itr.DT_INI_EXERC.dt.month <= 6))]

def obter_trimestre_4(dataframe_dfp, dataframe_acm3):
    # filtrar os informes com data inicial em jan e data de dim em dez - df acm4
    df_acm4 = dataframe_dfp.loc[lambda dataframe_dfp: (dataframe_dfp.DT_REFER.dt.month == 12) & (dataframe_dfp.DT_INI_EXERC.dt.month < 10) &
                             (dataframe_dfp.DT_REFER.dt.year == dataframe_dfp.DT_INI_EXERC.dt.year)]
    # fazer o merge entre df acumulado 4 e df acumulado 3, mantendo o df acumulado 4 - df trim4
    df_trim4 = pd.merge(df_acm4, dataframe_acm3, how='left', on=['CD_CVM', 'CD_CONTA', 'ANO'], suffixes=['_acm4','_acm3'])
    # preencher valores VL_CONTA ausentes com 0, informes que não foram enviados por Cias até o trimestre 3
    df_trim4.VL_CONTA_acm3.fillna(value=0, inplace=True)
    # calcular o valor do trimestre 4, fazendo a subtração entre acumulado anual e o acumulado do trimestre 3
    df_trim4['RESULTADO'] = df_trim4['VL_CONTA_acm4'] - df_trim4['VL_CONTA_acm3']
    # deletar colunas desnecessárias
    df_trim4 = df_trim4.drop(['VL_CONTA_acm4','CNPJ_CIA_acm3', 'DT_REFER_acm3', 'DENOM_CIA_acm3', 'GRUPO_DFP_acm3', 
                 'MOEDA_acm3', 'ESCALA_MOEDA_acm3', 'ORDEM_EXERC_acm3', 'DT_INI_EXERC_acm3', 'DT_FIM_EXERC_acm3', 
                 'DS_CONTA_acm3', 'VL_CONTA_acm3', 'ST_CONTA_FIXA_acm3','SETOR_ATIV_acm3'], axis=1)
    # renomear as colunas para a concatenação com o df trim123
    return df_trim4.rename(columns = {'CNPJ_CIA_acm4':'CNPJ_CIA', 'DT_REFER_acm4':'DT_REFER', 
                             'DENOM_CIA_acm4':'DENOM_CIA', 'GRUPO_DFP_acm4':'GRUPO_DFP', 'MOEDA_acm4':'MOEDA', 
                             'ESCALA_MOEDA_acm4':'ESCALA_MOEDA', 'ORDEM_EXERC_acm4':'ORDEM_EXERC', 
                             'DT_INI_EXERC_acm4':'DT_INI_EXERC', 'DT_FIM_EXERC_acm4':'DT_FIM_EXERC', 
                             'DS_CONTA_acm4':'DS_CONTA', 'ST_CONTA_FIXA_acm4':'ST_CONTA_FIXA', 
                             'SETOR_ATIV_acm4':'SETOR_ATIV', 'RESULTADO':'VL_CONTA'})

def padronizacao_valor_conta(dataframe_dre):
    # PADRONIZAÇÃO DA COLUNA VL_CONTA
    # dividir por 1000 os valores em que a escala moeda é unidade
    dataframe_dre['VL_CONTA'] = dataframe_dre.apply(lambda x: x.VL_CONTA/1000 if x.ESCALA_MOEDA == 'UNIDADE' 
                                         else x.VL_CONTA, axis = 1)
    
    # renomear os dados da coluna escala moeda para o valor mil
    dataframe_dre['ESCALA_MOEDA'] = dataframe_dre.ESCALA_MOEDA.replace('UNIDADE', 'MIL')
    return dataframe_dre


# faz a limpeza dos datasets cad_cia, dre_itr e dre_dfp e retorna o dataset obinvest
def criar_dataset_obinvest():
    # carrega os datasets 
    df_cia, df_itr, df_dfp = carregar_datasets()

    #adicionar dados de Ano e Trimestres
    df_itr = adiciona_ano(df_itr)
    df_dfp = adiciona_ano(df_dfp)
    
    #Tratando dados de companhia
    df_cia = trata_dados_companhias(df_cia)
    
    # fazer merge entre cadastro e informe itr, obtendo o df itr com setor - dfs cad_cia e itr
    df_itr_setor = merge_dre_setor(df_itr, df_cia)

    #Filtrando dados dos 3 primeiros trimestres e adicionando os campos de ano e 
    df_trim123 = obter_trimestres_123(df_itr_setor)

    # OBTENÇÃO DO DF ACUMULADO TRIMESTRE 3
    df_acm3 = obter_acumulado_trim3(df_itr_setor)
    
    # FATIAMENTO DO DF DFP
    # filtrar últimos informes anuais - df dfp
    df_dfp = filtrarUltimoExerc(df_dfp)
    
    # fazer merge entre cadastro e informe itr, obtendo o df dfp com setor - dfs cad_cia e dfp
    df_dfp_setor = merge_dre_setor(df_dfp, df_cia)
    
    # OBTENÇÃO DO DF ACUMULADO TRIMESTRE 4
    df_trim4 = obter_trimestre_4(df_dfp_setor, df_acm3)
    
    # CRIAÇÃO DO DF OBINVEST
    # criar o df obinvest, concatenando os informes referentes aos 4 trimestres
    df_obinvest = pd.concat([df_trim123, df_trim4])
    
    # comparar registros antes e depois
    print('Antes: Linhas = {0} | Colunas = {1}'.format(df_obinvest.shape[0], df_obinvest.shape[1]))
    
    # deletar os registros duplicados
    df_obinvest.drop_duplicates(subset = ['CNPJ_CIA', 'CD_CVM', 'CD_CONTA', 'DT_INI_EXERC', 'DT_FIM_EXERC', 'ANO'], 
                              keep = 'last', inplace=True)
    
    print('Depois: Linhas = {0} | Colunas = {1}'.format(df_obinvest.shape[0], df_obinvest.shape[1]))
    
    #Adicionar dado de trimestre
    df_obinvest = adiciona_trimestre(df_obinvest)
    
    #padronizacao valores contas
    return padronizacao_valor_conta(df_obinvest)
    #return df_obinvest

## **Proveniência de Dados**

In [176]:
import sys, subprocess, datetime
from prov.model import ProvDocument, Namespace
from prov.dot import prov_to_dot
from IPython.display import Image

def gerar_saidas_proveniencia(documento_prov):
    entity = "DRE-CVM-PROV"
    #Generating the outup - a  Provenance Graph
    dot = prov_to_dot(documento_prov)
    graph = entity+".png"
    dot.write_png(graph)

    #Generating the Serialization - Output XML
    documento_prov.serialize(entity + ".xml", format='xml') 

    #Generating the Serialization - Output Turtle
    documento_prov.serialize(entity + ".ttl", format='rdf', rdf_format='ttl')

def adicionar_namespaces(documento_prov):
    # Declaring namespaces for various prefixes used in the excution of Randon Walk Experiment
    documento_prov.add_namespace('foaf', 'http://xmlns.com/foaf/0.1/')
    documento_prov.add_namespace('prov', 'http://www.w3.org/ns/prov#')
    documento_prov.add_namespace('void', 'http://vocab.deri.ie/void#')
    documento_prov.add_namespace('ufrj', 'https://www.ufrj.br')
    documento_prov.add_namespace('cvm', 'https://www.gov.br/cvm/pt-br')
    documento_prov.add_namespace('cvm-cademp', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/')
    documento_prov.add_namespace('cvm-dre-itr-2021-zip', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    return documento_prov

def criar_agentes(documento_prov):
    
    #criando agentes
    dagnts={}
    dagnts["ag-cvm"] = documento_prov.agent("cvm:CVM", {"prov:type":"prov:Organization", "foaf:name":"Comissão de Valores Mobiliários"})
    dagnts["ag-ufrj"] = documento_prov.agent("ufrj:UFRJ", {"prov:type":"prov:Organization", "foaf:name":"Universidade Federal do Rio de Janeiro"})
    dagnts["ag-ppgi"] = documento_prov.agent("ufrj:PPGI", {"prov:type":"prov:Organization", "foaf:name":"Programa de Pós Graduação em Informática"})
    dagnts["ag-mai712"] = documento_prov.agent("ufrj:MAI712", {"prov:type":"prov:Organization", "foaf:name":"Disciplina de Fundamentos de Ciências de Dados"})
    dagnts["ag-grupo2"] = documento_prov.agent("ufrj:GRUPO2", {"prov:type":"prov:Organization", "foaf:name":"Grupo 02 para o trabalho final"})
    dagnts["ag-aluno-gil"] = documento_prov.agent("ufrj:Saulo", {"prov:type":"prov:Person", "foaf:name":"Gilberto Gil Fidelis Gomes Passos", "foaf:mbox":"gilbertogilfgp@gmail.com"})
    dagnts["ag-aluno-saulo"] = documento_prov.agent("ufrj:Gil", {"prov:type":"prov:Person", "foaf:name":"Saulo Andrade Almeida", "foaf:mbox":"sauloandrade@gmail.com"})
    dagnts["ag-aluno-valquire"] = documento_prov.agent("ufrj:Valquire", {"prov:type":"prov:Person", "foaf:name":"Valquire da Silva de Jesus", "foaf:mbox":"valquirej@gmail.com"})
    return dagnts

def associar_agentes_ufrj(dicionario_agentes):
    dicionario_agentes["ag-ppgi"].actedOnBehalfOf(dicionario_agentes["ag-ufrj"])
    dicionario_agentes["ag-mai712"].actedOnBehalfOf(dicionario_agentes["ag-ppgi"])
    dicionario_agentes["ag-grupo2"].actedOnBehalfOf(dicionario_agentes["ag-mai712"])
    dicionario_agentes["ag-aluno-gil"].actedOnBehalfOf(dicionario_agentes["ag-grupo2"])
    dicionario_agentes["ag-aluno-saulo"].actedOnBehalfOf(dicionario_agentes["ag-grupo2"])
    dicionario_agentes["ag-aluno-valquire"].actedOnBehalfOf(dicionario_agentes["ag-grupo2"])

def criar_atividades_iniciais(documento_prov):
    #criando atividades
    dativs={}
    dativs["at-create-ds"] = documento_prov.activity("cvm:create-dataset")
    return dativs
    
def criar_entidades_iniciais(documento_prov):
    #criando atividades
    dents={}
    dents["ent-cademp-ds"] = documento_prov.entity('cvm-cademp:cad_cia_aberta.csv', {'prov:label': 'Dataset com dados cadastrais das empresas listadas na CVM', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr"] = documento_prov.entity('cvm:dre-itr', {'prov:label': 'Documento que representa o conceito de DREs do tipo Trimestral', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp"] = documento_prov.entity('cvm:dre-dfp', {'prov:label': 'Documento que representa o conceito de DREs do tipo Anual', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2021-zip"] = documento_prov.entity('cvm-dre-itr-2021-zip:itr_cia_aberta_2021.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2021', 'prov:type': 'foaf:Document'})
        
    return dents
    
def associar_agentes_atividades_entidades_iniciais(documento_prov, dicionario_agentes, 
                                                   dicionario_atividades, dicionario_entidades):
    
    #Associar a atividade de gerar dataset com o agente CVM
    documento_prov.wasAssociatedWith(dicionario_atividades["at-create-ds"], 
                                     dicionario_agentes["ag-cvm"])
    
    #Associando os datasets com a atividade de gerar dataset da CVM
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-cademp-ds"], 
                                  dicionario_atividades["at-create-ds"])
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitr"], 
                                  dicionario_atividades["at-create-ds"])    
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dredfp"], 
                                  dicionario_atividades["at-create-ds"])
    
    #Associando ZIPs das DREs ITR e DFP com as entidaes de Datasets genericos
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2021-zip"], 
                                  dicionario_entidades["ent-dreitr"])


def inicializaProveniencia():
    # Creating an empty provenance document
    doc_prov = ProvDocument()

    #criando os namespaces do documento de proveniencia
    doc_prov = adicionar_namespaces(doc_prov)
    
    #criando agentes
    agentes_dic = criar_agentes(doc_prov)
    
    #criando as hierarquia de agentes
    associar_agentes_ufrj(agentes_dic)
    
    #criando atividades iniciais
    atividades_dic = criar_atividades_iniciais(doc_prov)
    
    #criando entidades iniciais
    entidades_dic = criar_entidades_iniciais(doc_prov)
    
    #associacoes inicias da proveniencia
    associar_agentes_atividades_entidades_iniciais(doc_prov, agentes_dic, atividades_dic, entidades_dic)
    
    #criando entidades iniciais
    return doc_prov, agentes_dic, atividades_dic, entidades_dic

## **Análise de Dados**
### **Análise Inicial dos Dados**

In [177]:
def analiseInicialDataset(dataframe_obinvest):
    #print(dataframe_obinvest.info())
    print(dataframe_obinvest.shape)
    #print(dataframe_obinvest.isnull().sum())
    #print(dataframe_obinvest.describe())
    #print(dataframe_obinvest.columns)
    #print(dataframe_obinvest.DT_REFER.value_counts())
    #print(dataframe_obinvest.ESCALA_MOEDA.value_counts())

### **Análise Estatística dos Dados**

In [178]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff

def analiseEstatisticaDataset(dataFrameObinvest):
    print('Rodando experimento de dataset estatisticos')
    #sns.displot(dataFrameObinvest['VL_CONTA'], bins=30)
    #fig1 = px.scatter(dataFrameObinvest, x = dataFrameObinvest.CD_CONTA.index, y = dataFrameObinvest.ANO, 
    #                  color=dataFrameObinvest.CD_CONTA)
    #fig1.show()
    contas = dataFrameObinvest.groupby(['CD_CONTA', 'SETOR_ATIV', 'ANO', 'TRIMESTRE'])['VL_CONTA'].sum()
    print(contas)
    print(dataFrameObinvest.columns)
    #print(dataFrameObinvest.head())
    
    #resposta=dataFrameObinvest.loc[dataFrameObinvest['DT_INI_EXERC']=='2019-07-01']
    #print(resposta)

## **Execução Principal**

In [179]:
def executaExperimento():
    doc_prov, dic_agentes , dic_atividades, dic_entidades = inicializaProveniencia()
    df_obinvest = criar_dataset_obinvest()
    analiseInicialDataset(df_obinvest)
    analiseEstatisticaDataset(df_obinvest)
    gerar_saidas_proveniencia(doc_prov)

In [180]:
def main():
    #if(checkingEnvironmentVersions(True)):
        executaExperimento()
        
main() # condição suspensa para poder rodar no notebook local

#Image("DRE-CVM-PROV.png")

Antes: Linhas = 133703 | Colunas = 16
Depois: Linhas = 108569 | Colunas = 16
(108569, 17)
Rodando experimento de dataset estatisticos
CD_CONTA    SETOR_ATIV                                  ANO   TRIMESTRE
3.01        Agricultura (Açúcar, Álcool e Cana)         2020  1            4.644205e+06
                                                              2            9.718834e+06
                                                              3            1.358116e+07
                                                              4            5.366045e+06
                                                        2021  1            6.830146e+06
                                                                               ...     
3.99.02.07  Emp. Adm. Part. - Intermediação Financeira  2021  2            4.998000e-02
                                                              3            1.176100e-01
                                                              4            5.157000e-02
3.